Create a model using built-in library from Pytorch.
This code closely follows nn_tutorial notebook.

In [1]:
import csv, math, os, pickle, torch
import torch.nn.functional as F
import IPython.display as ipd
from torch import nn
from torch import optim

global chroma_shape, epochs, train_bs, validate_bs, lr, n_class

## Set the number of classes
n_class = 9

In [2]:
### Count the proportions of correct predicton
def accuracy(out, yb):
    ##get the index with the max
    preds = torch.argmax(out, dim = 1)
    return (preds == yb).float().mean()


In [3]:
##Load record voice that is saved at temp_record.pkl

att_file = open(r'temp/temp_record.pkl', 'rb')
x_validate = pickle.load(att_file)  
att_file.close()

print('You are training this audio file: ')
ipd.Audio('temp/temp_sound_record.mp3', rate = 22050)

You are training this audio file: 


In [4]:
### If there is only one file, len(x_validate) = 13
### Unsafe if input is actually 13
if len(x_validate) == 13:
    chroma_shape = x_validate.shape
else:
    chroma_shape = x_validate[0].shape

In [5]:
##get the size of the input
n_validate = chroma_shape[0]
loss_func = F.cross_entropy

In [6]:
### the definition of this class must be the exact same as the one defined in model_CNN_train.
### Create one class to share and call from there?

class SoundRecognition_CNN(nn.Module):
  
    def __init__(self, dropout_rate):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        
        self.dropout = nn.Dropout(p = dropout_rate)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(16*3, n_class)

    def forward(self, xb):
        
        xb = xb.view(-1, 1, chroma_shape[0], chroma_shape[1])
        
        xb = self.dropout(F.relu(self.conv1(xb)))
        xb = self.dropout(F.relu(self.conv2(xb)))
        xb = self.dropout(F.relu(self.conv3(xb)))
        
        xb = self.pool(xb)
        
        ## reshape for fully connected
        xb = xb.view(-1, 16*3*116)
        xb = self.fc1(xb)
        return xb.view(-1, xb.size(1))

## Get the model and optim object that will be used to update model parameters
def get_model(dropout_rate, weight_decay, lr):
    model = SoundRecognition_CNN(dropout_rate)
    return model, optim.Adam(model.parameters(), weight_decay = weight_decay, lr = lr)

In [7]:
## convert number representation into interpretable outcome
def classify_raw_out(pred):
    
    p = torch.argmax(pred, dim = 1)
    class_map = ['c', 'ch', 'q', 's', 'sh', 'x', 'z', 'zh'] 
        
    return class_map[p]   

In [8]:
### Read in a trained model [there are currently 12 models we are considering]
audio_dir = '/Users/panchanok/Desktop/PyHack2019/PyHack2019/model8/'
model_files = audio_dir + 'mfcc_m_1000_dr0.5_wc0.0001_lr1e-06.pt'

In [9]:
model_files

'/Users/panchanok/Desktop/PyHack2019/PyHack2019/model8/mfcc_m_1000_dr0.5_wc0.0001_lr1e-06.pt'

In [10]:
# Model class must be defined somewhere
decision = []
raw = []
# for mf in model_files:
model = torch.load(model_files, map_location=torch.device('cpu'))
y = model(torch.tensor(x_validate))
raw.append(y)
decision.append(classify_raw_out(y))

//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.pooling.MaxPool2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set 

In [11]:
print('It thinks you said: ', decision)

It thinks you said:  ['z']
